In [3]:
import os

import numpy as np
from beartype.typing import Sequence


def _load(path) -> dict[str, int | float]:
    indices = np.load(os.path.join(path, "_fusion", "indices.npz"))
    trajectory = np.load(os.path.join(path, "_lidar", "pose.npz"))
    vel = np.diff(trajectory["pos"], axis=0)
    return {
        "n": indices['indices'].shape[0],
        "t": (trajectory['t'][-1] - trajectory['t'][0]) / 60 / 60,
        "d": np.sum(np.linalg.norm(vel, axis=1)) / 1000
    }


def _total(data: Sequence[dict[str, int | float]]) -> dict[str, int | float]:
    return {k: sum(x[k] for x in data) for k in data[0]}


def _report(traces: dict[str, dict[str, int | float]]) -> None:
    total = _total(list(traces.values()))
    print(f"{'total':18}{total['n']:8}{total['t']:8.02f}h{total['d']:8.02f}km")
    print("-" * (18 + 8 + 8 + 8 + len('hkm')))
    for n, s in traces.items():
        print(f"{n:18}{s['n']:8}{s['t']:8.02f}h{s['d']:8.02f}km")

In [6]:
path = "/data/radarhd2/outdoor"
ds = {
    trace: _load(os.path.join(path, trace))
    for trace in sorted(os.listdir(path))}
_report(ds)

total               371823   10.71h   54.45km
---------------------------------------------
baum                 15691    0.48h    2.30km
centre               14573    0.43h    2.12km
cmu.east             18027    0.51h    2.53km
cmu.north            14322    0.40h    2.01km
cmu.west             16824    0.47h    2.33km
craig.east           18519    0.53h    2.53km
craig.west           16865    0.48h    2.41km
ellsworth            16320    0.46h    2.35km
fifth                21590    0.61h    3.09km
forbes.east          14120    0.40h    1.94km
forbes.west          16397    0.47h    2.21km
museum               11024    0.32h    1.57km
oakland.central       9882    0.29h    1.39km
oakland.east         21892    0.64h    3.32km
oakland.north        20009    0.58h    3.10km
oakland.schf         15752    0.49h    2.55km
oakland.wpsbc        21054    0.60h    3.24km
pitt.central         16582    0.47h    2.39km
pitt.west            16090    0.49h    2.53km
schenley             12519    0.37

In [5]:
path = "/data/radarhd2/indoor"
ds = {
    building: _total([
        _load(os.path.join(path, building, dataset))
        for dataset in sorted(os.listdir(os.path.join(path, building)))])
    for building in sorted(os.listdir(path))}
_report(ds)

total               309726    8.92h   31.94km
---------------------------------------------
baker                16040    0.46h    1.62km
cfa                  13589    0.39h    1.32km
cic                  18540    0.53h    1.73km
doherty              15205    0.44h    1.64km
gates                28519    0.82h    2.95km
hamburg              21350    0.62h    2.18km
hunt                 11747    0.34h    1.14km
morrison              7765    0.23h    0.79km
nsh                  24675    0.71h    2.42km
porter               15119    0.43h    1.51km
posner               21658    0.61h    2.05km
roberts              11077    0.32h    1.21km
scaife               10041    0.29h    1.00km
scott                14704    0.42h    1.60km
tepper               30331    0.87h    3.31km
uc                   26968    0.79h    3.01km
wean                 22398    0.66h    2.45km
